Authors: A.Radhakrishnan, GFDL
Ack: Anderson Banihirwe for intake-esm updates, GFDL colleagues for data

In [1]:
import xarray as xr
import intake
import intake_esm
import sys
from dask_gateway import Gateway
import pandas as pd
pd.set_option("display.max_colwidth", None)
pd.options.display.max_rows = None

In [2]:
def launchDask(options):
    #TODO try, except
    gateway = Gateway()
    clusters = gateway.list_clusters()
    clusters
    if len(clusters) >0:
        cluster = gateway.connect(clusters[0].name)
        print("using existing connection")#or shut down and start afresh cluster.shutdown(). otherwise I dunno how to apply custom options
        cluster.shutdown()
        cluster = gateway.new_cluster(options)
    else:
        cluster = gateway.new_cluster(options)
    return(cluster)

In [3]:
gateway = Gateway()
options = gateway.cluster_options()
options.worker_memory=16
cluster = launchDask(options)


In [4]:

from distributed import Client
client = Client(cluster)
client

Client Scheduler: gateway://traefik-daskhub-dask-gateway.daskhub:80/daskhub.f70ae6bc486546c6a9d47b68cb8cae7a Dashboard: /services/dask-gateway/clusters/daskhub.f70ae6bc486546c6a9d47b68cb8cae7a/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [5]:
min_w = 2
max_w = 35

In [6]:
cluster.adapt(minimum=min_w, maximum=max_w)

esgf-world.json is the ESM collections spec file for the netCDF data in the S3 bucket esgf-world. 
The catalog is updated on an on-demand basis for now. 
You can refer to https://github.com/aradhakrishnanGFDL/gfdl-aws-analysis/tree/community/esm-collection-spec-examples for the most recent catalogs
More examples can be found in https://github.com/aradhakrishnanGFDL/gfdl-aws-analysis/tree/community/examples 


In [7]:
col_url = "https://cmip6-nc.s3.us-east-2.amazonaws.com/esgf-world.json" 

In [8]:
col = intake.open_esm_datastore(col_url)
esmcol_data = col.esmcol_data

In [9]:
def latest_version(cat):
    """
    input
    cat: esmdatastore
    output
    esmdatastore with latest DRS versions
    """
    latest_cat = cat.df.sort_values(by=['version','path']).drop_duplicates(['temporal subset','model','mip_table',
                                               'institute','variable','ensemble_member',
                                               'grid_label','experiment_id'],keep='last')
    return latest_cat

In [10]:
query_Omon_tauuo = dict(experiment_id=['abrupt-4xCO2','1pctCO2','historical'],
                 mip_table=['Omon'],
                 ensemble_member=["r1i1p1f1","r1i1p1f2"],
                 model=['CESM2','CNRM-CM6-1','UKESM1-0-LL','GFDL-ESM4'],
                 grid_label=['gn'],
                 variable=["tauuo"])
query_Omon_tauvo = dict(experiment_id=['abrupt-4xCO2','1pctCO2','historical'],
                 mip_table=['Omon'],
                 ensemble_member=["r1i1p1f1","r1i1p1f2"],
                 model=['CESM2','CNRM-CM6-1','UKESM1-0-LL','GFDL-ESM4'],
                 grid_label=['gn'],
                 variable=["tauvo"])
query_Omon_zos = dict(experiment_id=['abrupt-4xCO2','1pctCO2','historical'],
                 mip_table=['Omon'],
                 ensemble_member=["r1i1p1f1","r1i1p1f2"],
                 model=['CESM2','CNRM-CM6-1','UKESM1-0-LL','GFDL-ESM4'],
                 grid_label=['gn'],
                 variable=["zos"])
query_Ofx = dict(experiment_id=['1pctCO2','abrupt-4xCO2','historical'],
                 mip_table=['Ofx'],
                 ensemble_member=["r1i1p1f1","r1i1p1f2"],
                 model=['CESM2','CNRM-CM6-1','UKESM1-0-LL','GFDL-ESM4'],
                 grid_label=['gn'],
                 variable=["deptho"])

cat_Omon_tauuo = col.search(**query_Omon_tauuo)
cat_Omon_tauuo_lat = latest_version(cat_Omon_tauuo)

cat_Omon_tauvo = col.search(**query_Omon_tauvo)
cat_Omon_tauvo_lat = latest_version(cat_Omon_tauvo)

cat_Omon_zos = col.search(**query_Omon_zos)
cat_Omon_zos_lat = latest_version(cat_Omon_zos)


cat_Ofx = col.search(**query_Ofx)
cat_Ofx_lat = latest_version(cat_Ofx)



In [ ]:
#check back later 
query_Ofx_ukesm1 = dict(experiment_id=['piControl'],
                 mip_table=['Omon','Ofx'],
              #   ensemble_member=["r1i1p1f1","r1i1p1f2"],
                 model=['UKESM1-0-LL'],
                 grid_label=['gn'],
                 variable=["deptho"])

cat_subset_Ofx_ukesm1 = col.search(**query_Ofx_ukesm1)
cat_latest_Ofx_ukesm1 = latest_version(cat_subset_Ofx_ukesm1)

In [11]:
cat_Ofx_latest = intake.open_esm_datastore(cat_Ofx_lat,esmcol_data=esmcol_data)

cat_Omon_tauuo_latest = intake.open_esm_datastore(cat_Omon_tauuo_lat,esmcol_data=esmcol_data)
cat_Omon_tauvo_latest = intake.open_esm_datastore(cat_Omon_tauvo_lat,esmcol_data=esmcol_data)
cat_Omon_zos_latest = intake.open_esm_datastore(cat_Omon_zos_lat,esmcol_data=esmcol_data)


In [12]:
cat_Ofx_latest.df.groupby(['variable','model','experiment_id','version']).nunique()
#the temporal_subset count and path indicate the number of actual files/objects corresponding for each condition

project  institute  frequency  \
variable model      experiment_id version                                    
deptho   CESM2      1pctCO2       v20190425        1          1          0   
                    abrupt-4xCO2  v20190927        1          1          0   
                    historical    v20190308        1          1          0   
         CNRM-CM6-1 1pctCO2       v20180626        1          1          0   
                    abrupt-4xCO2  v20180705        1          1          0   
                    historical    v20180917        1          1          0   
         GFDL-ESM4  1pctCO2       v20180701        1          1          0   
                    abrupt-4xCO2  v20180701        1          1          0   
                    historical    v20190726        1          1          0   

                                             modeling_realm  mip_table  \
variable model      experiment_id version                                
deptho   CESM2      1pctCO2       v20190425               0          1   
                    abrupt-4xCO2  v20190927               0          1   
                    historical    v20190308               0          1   
         CNRM-CM6-1 1pctCO2       v20180626               0          1   
                    abrupt-4xCO2  v20180705               0          1   
                    historical    v20180917               0          1   
         GFDL-ESM4  1pctCO2       v20180701               0          1   
                    abrupt-4xCO2  v20180701               0          1   
                    historical    v20190726               0          1   

                                             ensemble_member  grid_label  \
variable model      experiment_id version                                  
deptho   CESM2      1pctCO2       v20190425                1           1   
                    abrupt-4xCO2  v20190927                1           1   
                    historical    v20190308                1           1   
         CNRM-CM6-1 1pctCO2       v20180626                1           1   
                    abrupt-4xCO2  v20180705                1           1   
                    historical    v20180917                1           1   
         GFDL-ESM4  1pctCO2       v20180701                1           1   
                    abrupt-4xCO2  v20180701                1           1   
                    historical    v20190726                1           1   

                                             temporal subset  path  
variable model      experiment_id version                           
deptho   CESM2      1pctCO2       v20190425                0     1  
                    abrupt-4xCO2  v20190927                0     1  
                    historical    v20190308                0     1  
         CNRM-CM6-1 1pctCO2       v20180626                0     1  
                    abrupt-4xCO2  v20180705                0     1  
                    historical    v20180917                0     1  
         GFDL-ESM4  1pctCO2       v20180701                0     1  
                    abrupt-4xCO2  v20180701                0     1  
                    historical    v20190726                0     1

In [13]:
cat_Omon_tauuo_latest.df.groupby(['variable','model','experiment_id','version']).nunique()
#CMIP6.NCAR.CESM2.1pctCO2.Omon

project  institute  frequency  \
variable model       experiment_id version                                    
tauuo    CESM2       1pctCO2       v20190425        1          1          1   
                     abrupt-4xCO2  v20190927        1          1          1   
                     historical    v20190308        1          1          1   
         CNRM-CM6-1  1pctCO2       v20180626        1          1          1   
                     abrupt-4xCO2  v20180705        1          1          1   
                     historical    v20180917        1          1          1   
         GFDL-ESM4   1pctCO2       v20180701        1          1          1   
                     abrupt-4xCO2  v20180701        1          1          1   
                     historical    v20190726        1          1          1   
         UKESM1-0-LL 1pctCO2       v20190701        1          1          1   
                     abrupt-4xCO2  v20190701        1          1          1   
                     historical    v20190627        1          1          1   

                                              modeling_realm  mip_table  \
variable model       experiment_id version                                
tauuo    CESM2       1pctCO2       v20190425               1          1   
                     abrupt-4xCO2  v20190927               1          1   
                     historical    v20190308               1          1   
         CNRM-CM6-1  1pctCO2       v20180626               1          1   
                     abrupt-4xCO2  v20180705               1          1   
                     historical    v20180917               1          1   
         GFDL-ESM4   1pctCO2       v20180701               1          1   
                     abrupt-4xCO2  v20180701               1          1   
                     historical    v20190726               1          1   
         UKESM1-0-LL 1pctCO2       v20190701               1          1   
                     abrupt-4xCO2  v20190701               1          1   
                     historical    v20190627               1          1   

                                              ensemble_member  grid_label  \
variable model       experiment_id version                                  
tauuo    CESM2       1pctCO2       v20190425                1           1   
                     abrupt-4xCO2  v20190927                1           1   
                     historical    v20190308                1           1   
         CNRM-CM6-1  1pctCO2       v20180626                1           1   
                     abrupt-4xCO2  v20180705                1           1   
                     historical    v20180917                1           1   
         GFDL-ESM4   1pctCO2       v20180701                1           1   
                     abrupt-4xCO2  v20180701                1           1   
                     historical    v20190726                1           1   
         UKESM1-0-LL 1pctCO2       v20190701                1           1   
                     abrupt-4xCO2  v20190701                1           1   
                     historical    v20190627                1           1   

                                              temporal subset  path  
variable model       experiment_id version                           
tauuo    CESM2       1pctCO2       v20190425                3     3  
                     abrupt-4xCO2  v20190927               18    18  
                     historical    v20190308                1     1  
         CNRM-CM6-1  1pctCO2       v20180626                1     1  
                     abrupt-4xCO2  v20180705                1     1  
                     historical    v20180917                1     1  
         GFDL-ESM4   1pctCO2       v20180701                8     8  
                     abrupt-4xCO2  v20180701                8     8  
                     historical    v20190726                9     9  
         UKESM1-0-LL 1pctCO2       v201

In [14]:
cat_Omon_tauvo_latest.df.groupby(['variable','model','experiment_id','version']).nunique()


project  institute  frequency  \
variable model       experiment_id version                                    
tauvo    CESM2       1pctCO2       v20190425        1          1          1   
                     abrupt-4xCO2  v20190927        1          1          1   
                     historical    v20190308        1          1          1   
         CNRM-CM6-1  1pctCO2       v20180626        1          1          1   
                     abrupt-4xCO2  v20180705        1          1          1   
                     historical    v20180917        1          1          1   
         GFDL-ESM4   1pctCO2       v20180701        1          1          1   
                     abrupt-4xCO2  v20180701        1          1          1   
                     historical    v20190726        1          1          1   
         UKESM1-0-LL 1pctCO2       v20190406        1          1          1   
                     abrupt-4xCO2  v20190406        1          1          1   
                     historical    v20190406        1          1          1   

                                              modeling_realm  mip_table  \
variable model       experiment_id version                                
tauvo    CESM2       1pctCO2       v20190425               1          1   
                     abrupt-4xCO2  v20190927               1          1   
                     historical    v20190308               1          1   
         CNRM-CM6-1  1pctCO2       v20180626               1          1   
                     abrupt-4xCO2  v20180705               1          1   
                     historical    v20180917               1          1   
         GFDL-ESM4   1pctCO2       v20180701               1          1   
                     abrupt-4xCO2  v20180701               1          1   
                     historical    v20190726               1          1   
         UKESM1-0-LL 1pctCO2       v20190406               1          1   
                     abrupt-4xCO2  v20190406               1          1   
                     historical    v20190406               1          1   

                                              ensemble_member  grid_label  \
variable model       experiment_id version                                  
tauvo    CESM2       1pctCO2       v20190425                1           1   
                     abrupt-4xCO2  v20190927                1           1   
                     historical    v20190308                1           1   
         CNRM-CM6-1  1pctCO2       v20180626                1           1   
                     abrupt-4xCO2  v20180705                1           1   
                     historical    v20180917                1           1   
         GFDL-ESM4   1pctCO2       v20180701                1           1   
                     abrupt-4xCO2  v20180701                1           1   
                     historical    v20190726                1           1   
         UKESM1-0-LL 1pctCO2       v20190406                1           1   
                     abrupt-4xCO2  v20190406                1           1   
                     historical    v20190406                1           1   

                                              temporal subset  path  
variable model       experiment_id version                           
tauvo    CESM2       1pctCO2       v20190425                3     3  
                     abrupt-4xCO2  v20190927               18    18  
                     historical    v20190308                1     1  
         CNRM-CM6-1  1pctCO2       v20180626                1     1  
                     abrupt-4xCO2  v20180705                1     1  
                     historical    v20180917                1     1  
         GFDL-ESM4   1pctCO2       v20180701                8     8  
                     abrupt-4xCO2  v20180701                8     8  
                     historical    v20190726                9     9  
         UKESM1-0-LL 1pctCO2       v201

In [15]:
cat_Omon_zos_latest.df.groupby(['variable','model','experiment_id','version']).nunique()


project  institute  frequency  \
variable model       experiment_id version                                    
zos      CESM2       1pctCO2       v20190425        1          1          1   
                     abrupt-4xCO2  v20190927        1          1          1   
                     historical    v20190308        1          1          1   
         CNRM-CM6-1  1pctCO2       v20180626        1          1          1   
                     abrupt-4xCO2  v20180705        1          1          1   
                     historical    v20180917        1          1          1   
         GFDL-ESM4   1pctCO2       v20180701        1          1          1   
                     abrupt-4xCO2  v20180701        1          1          1   
                     historical    v20190726        1          1          1   
         UKESM1-0-LL 1pctCO2       v20190701        1          1          1   
                     abrupt-4xCO2  v20190701        1          1          1   
                     historical    v20190627        1          1          1   

                                              modeling_realm  mip_table  \
variable model       experiment_id version                                
zos      CESM2       1pctCO2       v20190425               1          1   
                     abrupt-4xCO2  v20190927               1          1   
                     historical    v20190308               1          1   
         CNRM-CM6-1  1pctCO2       v20180626               1          1   
                     abrupt-4xCO2  v20180705               1          1   
                     historical    v20180917               1          1   
         GFDL-ESM4   1pctCO2       v20180701               1          1   
                     abrupt-4xCO2  v20180701               1          1   
                     historical    v20190726               1          1   
         UKESM1-0-LL 1pctCO2       v20190701               1          1   
                     abrupt-4xCO2  v20190701               1          1   
                     historical    v20190627               1          1   

                                              ensemble_member  grid_label  \
variable model       experiment_id version                                  
zos      CESM2       1pctCO2       v20190425                1           1   
                     abrupt-4xCO2  v20190927                1           1   
                     historical    v20190308                1           1   
         CNRM-CM6-1  1pctCO2       v20180626                1           1   
                     abrupt-4xCO2  v20180705                1           1   
                     historical    v20180917                1           1   
         GFDL-ESM4   1pctCO2       v20180701                1           1   
                     abrupt-4xCO2  v20180701                1           1   
                     historical    v20190726                1           1   
         UKESM1-0-LL 1pctCO2       v20190701                1           1   
                     abrupt-4xCO2  v20190701                1           1   
                     historical    v20190627                1           1   

                                              temporal subset  path  
variable model       experiment_id version                           
zos      CESM2       1pctCO2       v20190425                3     3  
                     abrupt-4xCO2  v20190927               18    18  
                     historical    v20190308                1     1  
         CNRM-CM6-1  1pctCO2       v20180626                1     1  
                     abrupt-4xCO2  v20180705                1     1  
                     historical    v20180917                1     1  
         GFDL-ESM4   1pctCO2       v20180701                8     8  
                     abrupt-4xCO2  v20180701                8     8  
                     historical    v20190726                9     9  
         UKESM1-0-LL 1pctCO2       v201

In [22]:
dset_dict_deptho = cat_Ofx_latest.to_dataset_dict(storage_options=dict(anon=True),cdf_kwargs={'decode_times': False, 'chunks': {}})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institute.model.experiment_id.mip_table'


In [18]:
dset_dict_tauuo = cat_Omon_tauuo_latest.to_dataset_dict(storage_options=dict(anon=True),cdf_kwargs={'decode_times': False, 'chunks': {}})




--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institute.model.experiment_id.mip_table'


In [39]:
cat_Omon_tauuo_latest.df.loc[(cat_Omon_tauuo_latest.df['model'] =='CNRM-CM6-1') & (cat_Omon_tauuo_latest.df['variable'] =='tauuo')] # & (cat_Omon_tauuo_latest.df['experiment_id'] =='1pctCO2')]

,project,institute,model,experiment_id,frequency,modeling_realm,mip_table,ensemble_member,grid_label,variable,temporal subset,version,path
0,CMIP6,CNRM-CERFACS,CNRM-CM6-1,1pctCO2,mon,ocean,Omon,r1i1p1f2,gn,tauuo,185001-199912,v20180626,s3://esgf-world/CMIP6/CMIP/CNRM-CERFACS/CNRM-CM6-1/1pctCO2/r1i1p1f2/Omon/tauuo/gn/v20180626/tauuo_Omon_CNRM-CM6-1_1pctCO2_r1i1p1f2_gn_185001-199912.nc
1,CMIP6,CNRM-CERFACS,CNRM-CM6-1,abrupt-4xCO2,mon,ocean,Omon,r1i1p1f2,gn,tauuo,185001-199912,v20180705,s3://esgf-world/CMIP6/CMIP/CNRM-CERFACS/CNRM-CM6-1/abrupt-4xCO2/r1i1p1f2/Omon/tauuo/gn/v20180705/tauuo_Omon_CNRM-CM6-1_abrupt-4xCO2_r1i1p1f2_gn_185001-199912.nc
2,CMIP6,CNRM-CERFACS,CNRM-CM6-1,historical,mon,ocean,Omon,r1i1p1f2,gn,tauuo,185001-201412,v20180917,s3://esgf-world/CMIP6/CMIP/CNRM-CERFACS/CNRM-CM6-1/historical/r1i1p1f2/Omon/tauuo/gn/v20180917/tauuo_Omon_CNRM-CM6-1_historical_r1i1p1f2_gn_185001-201412.nc


In [19]:
dset_dict_tauvo = cat_Omon_tauvo_latest.to_dataset_dict(storage_options=dict(anon=True),cdf_kwargs={'decode_times': False, 'chunks': {}})



--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institute.model.experiment_id.mip_table'


In [20]:
dset_dict_zos = cat_Omon_zos_latest.to_dataset_dict(storage_options=dict(anon=True),cdf_kwargs={'decode_times': False, 'chunks': {}})



--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institute.model.experiment_id.mip_table'


In [ ]:
dset_dict_tauuo

In [24]:
ds_deptho_example = dset_dict_deptho["CMIP6.CNRM-CERFACS.CNRM-CM6-1.1pctCO2.Ofx"]

In [26]:
ds_deptho_example.deptho

<xarray.DataArray 'deptho' (ensemble_member: 1, lev: 75, y: 294, x: 362)>
dask.array<broadcast_to, shape=(1, 75, 294, 362), dtype=float32, chunksize=(1, 75, 294, 362), chunktype=numpy.ndarray>
Coordinates:
    lat              (y, x) float64 dask.array<chunksize=(294, 362), meta=np.ndarray>
    lon              (y, x) float64 dask.array<chunksize=(294, 362), meta=np.ndarray>
  * lev              (lev) float64 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
  * ensemble_member  (ensemble_member) <U8 'r1i1p1f2'
Dimensions without coordinates: y, x
Attributes:
    standard_name:     sea_floor_depth_below_geoid
    long_name:         Sea Floor Depth Below Geoid
    units:             m
    online_operation:  once
    cell_methods:      area: mean
    description:       Ocean bathymetry. Reported here is the sea floor depth...
    history:           none
    cell_measures:     area: areacello

In [32]:
ds_tauuo_example = dset_dict_tauuo["CMIP6.MOHC.UKESM1-0-LL.abrupt-4xCO2.Omon"]

In [33]:
ds_tauuo_example.tauuo

<xarray.DataArray 'tauuo' (ensemble_member: 1, time: 1800, j: 330, i: 360)>
dask.array<broadcast_to, shape=(1, 1800, 330, 360), dtype=float32, chunksize=(1, 1200, 330, 360), chunktype=numpy.ndarray>
Coordinates:
  * time             (time) float64 15.0 45.0 75.0 ... 5.396e+04 5.398e+04
  * j                (j) int32 0 1 2 3 4 5 6 7 ... 323 324 325 326 327 328 329
  * i                (i) int32 0 1 2 3 4 5 6 7 ... 353 354 355 356 357 358 359
    latitude         (j, i) float32 dask.array<chunksize=(330, 360), meta=np.ndarray>
    longitude        (j, i) float32 dask.array<chunksize=(330, 360), meta=np.ndarray>
  * ensemble_member  (ensemble_member) <U8 'r1i1p1f2'
Attributes:
    standard_name:  surface_downward_x_stress
    long_name:      Surface Downward X Stress
    comment:        This is the stress on the liquid ocean from overlying atm...
    units:          N m-2
    original_name:  mo: mask_copy((variable_name: tauuo), (variable_name: mas...
    cell_methods:   time: mean

In [34]:
ds_tauvo_example = dset_dict_tauvo["CMIP6.MOHC.UKESM1-0-LL.abrupt-4xCO2.Omon"]

In [35]:
ds_tauvo_example.tauvo

<xarray.DataArray 'tauvo' (ensemble_member: 1, time: 1800, j: 330, i: 360)>
dask.array<broadcast_to, shape=(1, 1800, 330, 360), dtype=float32, chunksize=(1, 1200, 330, 360), chunktype=numpy.ndarray>
Coordinates:
  * time             (time) float64 15.0 45.0 75.0 ... 5.396e+04 5.398e+04
  * j                (j) int32 0 1 2 3 4 5 6 7 ... 323 324 325 326 327 328 329
  * i                (i) int32 0 1 2 3 4 5 6 7 ... 353 354 355 356 357 358 359
    latitude         (j, i) float32 dask.array<chunksize=(330, 360), meta=np.ndarray>
    longitude        (j, i) float32 dask.array<chunksize=(330, 360), meta=np.ndarray>
  * ensemble_member  (ensemble_member) <U8 'r1i1p1f2'
Attributes:
    standard_name:  surface_downward_y_stress
    long_name:      Surface Downward Y Stress
    comment:        This is the stress on the liquid ocean from overlying atm...
    units:          N m-2
    original_name:  mo: mask_copy((variable_name: tauvo), (variable_name: mas...
    cell_methods:   time: mean

In [36]:
ds_zos_example = dset_dict_zos["CMIP6.MOHC.UKESM1-0-LL.abrupt-4xCO2.Omon"]

In [37]:
ds_zos_example.zos

<xarray.DataArray 'zos' (ensemble_member: 1, time: 1800, j: 330, i: 360)>
dask.array<broadcast_to, shape=(1, 1800, 330, 360), dtype=float32, chunksize=(1, 1200, 330, 360), chunktype=numpy.ndarray>
Coordinates:
  * time             (time) float64 15.0 45.0 75.0 ... 5.396e+04 5.398e+04
  * j                (j) int32 0 1 2 3 4 5 6 7 ... 323 324 325 326 327 328 329
  * i                (i) int32 0 1 2 3 4 5 6 7 ... 353 354 355 356 357 358 359
    latitude         (j, i) float32 dask.array<chunksize=(330, 360), meta=np.ndarray>
    longitude        (j, i) float32 dask.array<chunksize=(330, 360), meta=np.ndarray>
  * ensemble_member  (ensemble_member) <U8 'r1i1p1f2'
Attributes:
    standard_name:  sea_surface_height_above_geoid
    long_name:      Sea Surface Height Above Geoid
    comment:        This is the dynamic sea level, so should have zero global...
    units:          m
    original_name:  mo: (variable_name: zos)
    cell_methods:   area: mean where sea time: mean
    cell_measures:  area: areacello

In [ ]:
cluster.close()